In [1]:
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
from torch.utils.data import  DataLoader
import numpy as np
from gpytorch.kernels import Kernel
import pickle

from gpytorch.kernels import Kernel, RBFKernel

import torch.nn as nn
import logging
from itertools import product
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset,TensorDataset

from utils.HelperFunctions import *
from utils.Models import *
from utils.Kernels import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
param_grid = {
    'models':[Linear_Model_Of_Corregionalization,MultitaskGP],
    'kernels': [K_MS_with_Feat_Scaling,K_MS,K_Alpha_Beta,RBFKernel],
    'lrs': torch.log(torch.logspace(0.003,0.3,5)).tolist(),
    'gammas':torch.linspace(0.2,0.8,5).tolist(),
    'STEP_SIZEs':torch.linspace(10,100,5).tolist()
}


In [3]:
torch.linspace(0.2,0.8,5).tolist()

[0.20000000298023224,
 0.3500000238418579,
 0.5,
 0.6499999761581421,
 0.800000011920929]

# Global variables and configuration

In [55]:
# 创建一个logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# 创建文件处理器，写入日志文件
fh = logging.FileHandler('app.log')
fh.setLevel(logging.DEBUG)

# 创建控制台处理器，输出到控制台
ch = logging.StreamHandler()

# 设置日志格式
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

# 将处理器添加到logger中
# 创建控制台处理器，输出到控制台
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

if not any(isinstance(handler, logging.StreamHandler) for handler in logger.handlers):
    logger.addHandler(fh)
    logger.addHandler(ch)
    print("StreamHandler added")
else:
    print("StreamHandler already exists")

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available!")
USE_TOY_DATASET=False

StreamHandler already exists
GPU is not available!


In [65]:
# X_path="data/X_df_toy.pkl"
# y_path="data/y_df_toy.pkl"

# X_path="data/full_shots_Shikonin/X_df.pkl"
# y_path="data/full_shots_Shikonin/y_df.pkl"
# X_domain_path="data/full_shots_Shikonin/X_domain_info.pkl"

Global=config()

# X_path="data/X_df_2_shots.pkl"
# y_path="data/y_df_2_shots.pkl"
# X_domain_path="data/X_domain_info_2_shots.pkl"

# X_path="data/2_shots_4_params_sigmoid/X_df.pkl"
# y_path="data/2_shots_4_params_sigmoid/y_df.pkl"
# X_domain_path="data/2_shots_4_params_sigmoid/X_domain_info.pkl"

X_path="data/8_shots/X_df.pkl"
y_path="data/8_shots/y_df.pkl"
X_domain_path="data/8_shots/X_domain_info.pkl"

# X_path="data/2_shots/X_df.pkl"
# y_path="data/2_shots/y_df.pkl"
# X_domain_path="data/2_shots/X_domain_info.pkl"
(X_train_tensor,X_D_train_tensor,y_train_tensor),(X_test_tensor,X_D_test_tensor,y_test_tensor)=load_dataset(X_path,y_path,X_domain_path=X_domain_path)

print(X_train_tensor.shape)
print(X_D_train_tensor.shape)
print(y_train_tensor.shape)

Global.NUM_CONC=y_train_tensor.shape[1]

Global.NUM_FEAT=X_train_tensor.shape[1]
Global.NUM_DOMAIN_FEAT=X_D_train_tensor.shape[1]
NUMS_DOMAIN, max_indices_row = torch.max(X_D_train_tensor, dim=0)
NUMS_DOMAIN.add_(1)


print(f'NUMS_DOMAIN: {NUMS_DOMAIN.item()}')
Global.NUMS_DOMAIN=NUMS_DOMAIN.long()
X_train_tensor = torch.cat((X_D_train_tensor, X_train_tensor), dim=1)
X_test_tensor = torch.cat((X_D_test_tensor, X_test_tensor), dim=1)
print(X_train_tensor.shape)
print(X_test_tensor.shape)

with open(X_path, 'rb') as f:
    X_df = pickle.load(f)


(153, 1084)
(153, 10)
(153, 1)
torch.Size([137, 1084])
torch.Size([137, 1])
torch.Size([137, 10])
NUMS_DOMAIN: 8.0
torch.Size([137, 1085])
torch.Size([16, 1085])


# Tests

In [66]:
Global.lr=0.1
Global.gamma=0.5
Global.STEP_SIZE=50
Global.NUMS_DOMAIN_FEATURE=1

In [67]:
params= [
    # [MultitaskGP, RBFKernel, 0.18, 55, 0.8],
    # [Linear_Model_Of_Corregionalization, RBFKernel, 0.1, 60, 0.8],
    # [MultitaskGP, K_MS, 0.18, 55, 0.65],
    [Linear_Model_Of_Corregionalization, K_Alpha_Beta,0.69, 33, 0.2],
    # [MultitaskGP, K_MS_with_Feat_Scaling, 0.18, 55,0.65],
]

In [68]:
X = torch.cat((X_train_tensor, X_test_tensor), dim=0)
y = torch.cat((y_train_tensor, y_test_tensor), dim=0)
dataset = TensorDataset(X, y)

# 定义 KFold，n_splits=10 表示 10-fold 交叉验证
k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# 用于存储每个 fold 的结果
fold_results = {}
m=[]
nlls=[]
for each_param in params:
        Global.lr=each_param[2]
        Global.STEP_SIZE=each_param[3]
        Global.gamma=each_param[4]
        for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
            logger.info('--------------------------------')
            logger.info(f'FOLD {fold+1}')

            
            # 创建训练和验证数据集
            train_subset = Subset(dataset, train_idx)
            val_subset = Subset(dataset, val_idx)
            
            # 创建数据加载器
            train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
            val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
            X_train_tensor,y_train_tensor=dataloader2tensor(train_loader)
            X_test_tensor,y_test_tensor=dataloader2tensor(val_loader)
            try:
                md,n=run_test(X_train_tensor,y_train_tensor,X_test_tensor,y_test_tensor,model=each_param[0],kernel=each_param[1],config=Global,logger=logger)
                m.append(md)
                nlls.append(n)
            except Exception as e:
                logger.error(f"发生了一个异常: {e}")
                continue


2024-09-06 17:30:36,416 - INFO - --------------------------------
2024-09-06 17:30:36,417 - INFO - FOLD 1
2024-09-06 17:30:36,419 - INFO - training starts for model <class '__main__.Linear_Model_Of_Corregionalization'> with kernel <class '__main__.K_Alpha_Beta'>; lr: 0.69; step_size:33; gamma:0.2
2024-09-06 17:30:36,429 - DEBUG - Completed


X_train_tensor shape: torch.Size([137, 1085])
y_train_tensor shape: torch.Size([137, 10])
X_train_tensor shape: torch.Size([16, 1085])
y_train_tensor shape: torch.Size([16, 10])


2024-09-06 17:30:43,535 - DEBUG - Iter 33/500 - Loss: 0.994
2024-09-06 17:30:50,710 - DEBUG - Iter 66/500 - Loss: -0.366
2024-09-06 17:30:57,406 - DEBUG - Iter 99/500 - Loss: -0.950
2024-09-06 17:31:03,999 - DEBUG - Iter 132/500 - Loss: -1.126
2024-09-06 17:31:10,418 - DEBUG - Iter 165/500 - Loss: -1.244
2024-09-06 17:31:16,901 - DEBUG - Iter 198/500 - Loss: -1.267
2024-09-06 17:31:16,901 - INFO - Early cut off at epoch 197 with loss of -1.2670520544052124
2024-09-06 17:31:17,309 - INFO - NLL: -1.0605; RMSE: 0.2341; NMSE: 1.1836
2024-09-06 17:31:17,311 - INFO - --------------------------------
2024-09-06 17:31:17,312 - INFO - FOLD 2
2024-09-06 17:31:17,314 - INFO - training starts for model <class '__main__.Linear_Model_Of_Corregionalization'> with kernel <class '__main__.K_Alpha_Beta'>; lr: 0.69; step_size:33; gamma:0.2
2024-09-06 17:31:17,322 - DEBUG - Completed


X_train_tensor shape: torch.Size([137, 1085])
y_train_tensor shape: torch.Size([137, 10])
X_train_tensor shape: torch.Size([16, 1085])
y_train_tensor shape: torch.Size([16, 10])


2024-09-06 17:31:24,059 - DEBUG - Iter 33/500 - Loss: 0.762
2024-09-06 17:31:30,470 - DEBUG - Iter 66/500 - Loss: -0.256
2024-09-06 17:31:36,892 - DEBUG - Iter 99/500 - Loss: -0.817
2024-09-06 17:31:43,333 - DEBUG - Iter 132/500 - Loss: -1.068
2024-09-06 17:31:49,819 - DEBUG - Iter 165/500 - Loss: -1.060
2024-09-06 17:31:56,226 - DEBUG - Iter 198/500 - Loss: -1.057
2024-09-06 17:31:56,227 - INFO - Early cut off at epoch 197 with loss of -1.0574140548706055
2024-09-06 17:31:56,636 - INFO - NLL: -0.6869; RMSE: 0.2151; NMSE: 1.0260
2024-09-06 17:31:56,638 - INFO - --------------------------------
2024-09-06 17:31:56,638 - INFO - FOLD 3
2024-09-06 17:31:56,640 - INFO - training starts for model <class '__main__.Linear_Model_Of_Corregionalization'> with kernel <class '__main__.K_Alpha_Beta'>; lr: 0.69; step_size:33; gamma:0.2
2024-09-06 17:31:56,651 - DEBUG - Completed


...

2024-09-06 17:35:33,012 - DEBUG - Iter 33/500 - Loss: 0.953
2024-09-06 17:35:39,506 - DEBUG - Iter 66/500 - Loss: -0.255
2024-09-06 17:35:46,022 - DEBUG - Iter 99/500 - Loss: -1.009
2024-09-06 17:35:52,635 - DEBUG - Iter 132/500 - Loss: -1.232
2024-09-06 17:35:59,024 - DEBUG - Iter 165/500 - Loss: -1.283
2024-09-06 17:36:05,603 - DEBUG - Iter 198/500 - Loss: -1.281
2024-09-06 17:36:05,603 - INFO - Early cut off at epoch 197 with loss of -1.2814234495162964
2024-09-06 17:36:06,035 - INFO - NLL: -1.0681; RMSE: 0.1981; NMSE: 0.9976
2024-09-06 17:36:06,038 - INFO - --------------------------------
2024-09-06 17:36:06,039 - INFO - FOLD 9
2024-09-06 17:36:06,041 - INFO - training starts for model <class '__main__.Linear_Model_Of_Corregionalization'> with kernel <class '__main__.K_Alpha_Beta'>; lr: 0.69; step_size:33; gamma:0.2
2024-09-06 17:36:06,054 - DEBUG - Completed


X_train_tensor shape: torch.Size([138, 1085])
y_train_tensor shape: torch.Size([138, 10])
X_train_tensor shape: torch.Size([15, 1085])
y_train_tensor shape: torch.Size([15, 10])


2024-09-06 17:36:12,481 - DEBUG - Iter 33/500 - Loss: 0.824
2024-09-06 17:36:18,991 - DEBUG - Iter 66/500 - Loss: -0.329
2024-09-06 17:36:25,472 - DEBUG - Iter 99/500 - Loss: -1.062
2024-09-06 17:36:31,972 - DEBUG - Iter 132/500 - Loss: -1.230
2024-09-06 17:36:38,415 - DEBUG - Iter 165/500 - Loss: -1.294
2024-09-06 17:36:38,417 - INFO - Early cut off at epoch 164 with loss of -1.2944279909133911
2024-09-06 17:36:38,820 - INFO - NLL: -0.9007; RMSE: 0.2395; NMSE: 1.1651
2024-09-06 17:36:38,822 - INFO - --------------------------------
2024-09-06 17:36:38,823 - INFO - FOLD 10
2024-09-06 17:36:38,825 - INFO - training starts for model <class '__main__.Linear_Model_Of_Corregionalization'> with kernel <class '__main__.K_Alpha_Beta'>; lr: 0.69; step_size:33; gamma:0.2
2024-09-06 17:36:38,835 - DEBUG - Completed


X_train_tensor shape: torch.Size([138, 1085])
y_train_tensor shape: torch.Size([138, 10])
X_train_tensor shape: torch.Size([15, 1085])
y_train_tensor shape: torch.Size([15, 10])


2024-09-06 17:36:45,328 - DEBUG - Iter 33/500 - Loss: 1.011
2024-09-06 17:36:51,880 - DEBUG - Iter 66/500 - Loss: -0.301
2024-09-06 17:36:58,356 - DEBUG - Iter 99/500 - Loss: -1.171
2024-09-06 17:37:04,853 - DEBUG - Iter 132/500 - Loss: -1.290
2024-09-06 17:37:11,477 - DEBUG - Iter 165/500 - Loss: -1.299
2024-09-06 17:37:11,478 - INFO - Early cut off at epoch 164 with loss of -1.2993133068084717
2024-09-06 17:37:11,885 - INFO - NLL: -0.8710; RMSE: 0.2460; NMSE: 1.1389


In [46]:
param_grid = {
    'models':[Linear_Model_Of_Corregionalization],
    'kernels': [RBFKernel],
    'lrs': [0.69 ],
    'gammas':[0.5],
    'STEP_SIZEs':[33]
}


param_combinations = list(product(*param_grid.values()))

In [47]:
for each_param in param_combinations:
    Global.lr=each_param[2]
    Global.gamma=each_param[3]
    Global.STEP_SIZE=each_param[4]
    run_test(X_train_tensor,y_train_tensor,X_test_tensor,y_test_tensor,model=each_param[0],kernel=each_param[1],config=Global)